In [1]:
import json
import pystac
import rioxarray
import geopandas as gpd
from pystac_client import Client
import numpy as np
import xarray
import rasterio

import earthpy.plot as ep
import matplotlib.pyplot as plt

from matplotlib.colors import ListedColormap

In [2]:
country = "Argentina"
province = "Santa Cruz"

provinces = gpd.read_file("./data/states_provinces/ne_10m_admin_1_states_provinces.shp")
# Find Santa Cruz in the dataset
scdf = provinces.loc[(provinces['name'] == province) & (provinces['admin'] == country)]
scdf = scdf.reset_index(drop=True)

In [3]:
santa_cruz_dict = scdf.iloc[0].geometry.__geo_interface__
santa_cruz_polygon = json.dumps(santa_cruz_dict)

----

In [4]:
api_url = "https://earth-search.aws.element84.com/v1"
client = Client.open(api_url)
collection = "sentinel-2-l2a"  # Sentinel-2, Level 2A, Cloud Optimized GeoTiffs (COGs)

In [5]:
search = client.search(
    collections=[collection],
    intersects=santa_cruz_polygon,
    #max_items=10,
    datetime="2020-11-01/2020-12-01",
    query=["eo:cloud_cover<20"]
)
print(search.matched())

236


In [6]:
items = search.item_collection()

In [ ]:
source_bands = []
for item in items:
    print("Getting item:",item)
    red = rioxarray.open_rasterio(item.assets["red"].href, masked=True, lock=False, chunks=(1, 4000, 4000))
    nir = rioxarray.open_rasterio(item.assets["nir08"].href, masked=True, lock=False, chunks=(1, 4000, 4000))
    
    red_matched = red.rio.reproject_match(nir)
    ndvi = (nir - red_matched)/ (nir + red_matched)
    
    source_bands.append(ndvi)